JW notes indicated by # JW 

# ngCASA API and Usage Examples

Radio interferometry data analysis applications and algorithms may be assembled from CNGI and ngCASA building blocks. A user may choose to implement their own analysis scripts, use a pre-packaged task similar to those in current CASA or embed ngCASA and CNGI methods in a production pipeline DAG.

Below is a (very) preliminary API for ngCASA along with a few examples of applications and algorithms that could form some user-level tasks with interative interfaces. 


## Imaging
Iterative Image reconstruction




### Imaging API

In [0]:
def make_imaging_weight(img_dataset,vis_dataset, weightpars):
    """
    Calculate imaging weights for the specified weighting algorithm and image definition 
    
    Options : Natural, Uniform, Robust, Radial
    Additional Options : UV-taper.
    
    Note : The 'mosweight' option may be implemented via the a linear-mosaic in-between the major and minor cycles.
    
    """
    # JW  make_imaging_weight does not need an input img_dataset or produce one. I have fleshed out this function.
    
    
def make_gridding_convolution_function(img_dataset, gridpars):
    """
    Calculate gridding convolution functions (GCF) as specified for standard, widefield and mosaic imaging.
    Construct a GCF cache (persistent or on-the-fly)
    
    Options : Choose a list of effects to include
            PSterm : Prolate-Spheroidal gridding kernel (anti-aliasing function)
            Aterm : Use PB model and Aperture Illumination Function per antenna to construct a GCF per baseline
                    ( Include support for Heterogeneous Arrays where Aterm is different per antenna
                      Include support for time-varying PB and AIF models. Rotation, etc.)
            Wterm : FT of Fresnel kernel per baseline

            ( Note : Pointing offsets are applied as phase-gradients during gridding/degridding )
            
      TBD : W-stack ? How ? 
            
    """  
    # JW I don't think the we should move phase-gradients and pointing offsets to gridding/degridding. 
    #Can they be stand alone functions?

# JW #####################################################
def apply_phase_gradient(vis_dataset, global_dataset, phase_gradient_parms, storage_parms):
    """
    
    Returns
    -------
    vis_dataset : xarray.core.dataset.Dataset        
    """  
    
def apply_pointing_correction(vis_dataset, global_dataset, pointing_correction_parms, storage_parms):
    """

    Returns
    -------
    vis_dataset : xarray.core.dataset.Dataset             
    """  
# The global_dataset has the field phasecenters and pointing tables
######################################################

# JW I think there should also be a function that just makes the grid
def make_grid(vis_dataset, grid_parms, storage_parms):
    """

    Returns
    -------
    vis_dataset : xarray.core.dataset.Dataset             
    """  

######################################################

def make_psf(img_dataset, vis_dataset, gridpars):
    """
    Form a Point Spread Function cube by gridding the imaging weights (with flags) and an inverse Fourier transform
    
    (A cube with 1 channel is a continuum image (nterms=1))   
    """
    ngcasa.imaging._make_grid()
 # JW It's the users responsibility to apply flags. The gridder only looks for nans.
    
def make_pb(img_dataset,vis_dataset,gridpars):
    """
    Construct a Primary Beam cube containing a weighted sum of primary beams
    Option 1 : Evaluate models directly onto the image (for common PBs)
    Option 2 : Inverse FT each gridding convolution function (for varying PBs)

    (A cube with 1 channel is a continuum image (nterms=1))
    """   

def make_residual_image(img_dataset, vis_dataset,gridpars, normpars):
    """
    Form a residual image cube by gridding and inverse FTing the data - model.
    Use a pre-specified gridding convolution function cache.

    (A cube with 1 channel is a continuum image (nterms=1))

    """   
    cngi.vis.uvsub() # do the subtraction of data-model or corrected-model, etc etc..
    ngcasa.imaging._make_grid() 
    cngi.image.fourier_transform()
    cngi.image.corr_to_stokes() 
    ngcasa.imaging._normalize(direction='reverse') # (1) Do the multiply/div by PB, accounting for masks.
# JW I think this should be a higher level function. Is there a need for cngi.vis.uvsub (it is just a dask numpy substraction)?
    
def make_image(img_dataset, vis_dataset,gridpars, arr_name):
    """
    Form an image cube by gridding and inverse FTing the data in the specified arr_name
    Use a pre-specified gridding convolution function cache.
    
    (A cube with 1 channel is a continuum image (nterms=1))
    """   
    ngcasa.imaging._make_grid() 
    cngi.image.fourier_transform()
# JW I have fleshed out this function.

        
def predict_modelvis_image(img_dataset, vis_dataset, gridpars, normpars, arr_name, incremental):     
    """
    Predict model visibilities from an input model image cube (units Jy/pixel) using
    a pre-specified gridding convolution function cache.
    Save the model visibilities in arr_name (default = 'MODEL')
    Optionally overwrite the model or add to existing model (incremental=T)
    
    (A input cube with 1 channel is a continuum image (nterms=1))

    """   
    ngcasa.imaging._normalize(direction='forward') # Apply PB models to go to flat-sky
    cngi.image.stokes_to_corr()
    cngi.image.fourier_transform()
    ngcasa.imaging._degrid()  

    
def predict_modelvis_component(img_dataset, vis_dataset, arr_name, incremental):
    """
    Predict model visibilities from a component list by analytical evaluation
    Apply PB models to the components prior to evaluation.
    Save the model visibilities in arr_name (default = 'MODEL')
    Optionally overwrite the model or add to existing model (incremental=T)

    """   

        
def make_mask(img_dataset,maskpars,interactive=False):
    """
    Make a region to identify a mask for use in deconvolution. 
    
    One or more of the following options are allowed
    - Supply a mask in the form of a cngi.image.region
    - Run an auto-masking algorithm to detect structure and define a cngi.image.region
    - Apply a pblimit based mask

    An existing deconvolution mask from img_dataset may either be included in the above, or ignored. 

    The output is a region (array?) in the img_dataset containing the intersection of all above regions 
    
    (Note : Interactive masking is handled separately at the application layer, editing the region directly). 

    """   
# JW I don't think interactive should be a parameter
    
def is_converged(img_dataset, iterpars, iter_rec):
    """
    An iteration controller for image reconstruction
    
    The current image set (residual, psf, model, etc) is evaluated against stopping criteria derived 
    from input parameters (iterpars) and the image set itself. 
    
    Step 1 : Derive stopping criteria
    
    - Merge explicit user-parameters in iterpars (niter,threshold,etc..) with criteria that are calculated from
      the imageset (psfsidelobelevel, cyclethreshold, N-sigma-based thresholds, mask-sensitive thresholds)
    - Calculate 'cycleniter' and 'cyclethreshold' to be used in Step 2. 
    
    Step 2 : Apply stopping criteria (as an ordered list)
       
    - Peak residual within the mask region for imagename.residual <= threshold
    - Total iters done >= niter
        
    
    Outputs : 
    
    (1) Return a dict {'cyclethreshold':xxx, 'cycleniter':xxx', 'stopcode':xxx} 
        to be used for the subsequent minor cycle and/or 
        to indicate that a stopping criterion was satisfied.  
        
    (2) Save or append-to a 'convergence history' dictionary. TBD : Where should this dict live ?  
        This content is what gets printed to logs, plotted in an interactive GUI, 
        and forms a long-lived record of how the image reconstruction proceeded.    
    
    """   
    return {}
# JW My suggestion is that is_converged returns the img_dataset with the convergence history list of dict included in the attribute section.

def deconvolve_point_clean(img_dataset, decpars):
    """
    An iterative solver to construct a model from an observed image(set) and psf(set).
    
    Sky model : Point source
    Algorithm : CLEAN (a greedy algorithm for chi-square minimization)
       
    Options : Hogbom, Clark
    
    Input : Requires an input cube (mfs is a cube with nchan=1)
    Output : Cube model image    
    """   
    
def deconvolve_multiterm_clean(img_dataset, decpars):
    """
    An iterative solver to construct a model from an observed image(set) and psf(set).
    
    Sky model : A (multi-term) linear combination of basis functions.
                Multi-scale : Basis functions are inverted tapered paraboloids
                Multi-scale MFS : Basis functions are Taylor polynomials in frequency
    

    Options : 
     - MS-Clean : Multi-scale CLEAN ( MS-MFS Clean with nterms=1 )
                  Input : Requires an input cube (mfs is a cube with nchan=1)
                  Output : Cube model image
                  
     - MS-MFS Clean : Wideband Imaging that solves for a set of Taylor coefficient maps.
                  Input : Multi-channel cube.
                  Output : Taylor coefficient maps, Spectral Index + Evaluation of the model to a Cube model image
                  
                  Step (1) cngi.image.cube_to_mfs()
                  Step (2) Implement the multi-term deconvolution algorithm
                  Step (3) cngi.image.mfs_to_cube()
     
    The special case of nscales=1 and nterms=1 is the same use-case as deconvolve_point_clean.
    """   
    
def deconvolve_adaptive_scale_pixel(img_dataset, decpars):
    """
    An iterative solver to construct a 2D mixed model from an observed image(set) and psf(set).
    
    Sky Model : A linear combination of 2D Gaussians
    Algorithm : Chi-square / TV minimization on atom parameters, with subspace selections.
       
    Options : Narrow-band, Wide-band
    
    Input : Requires an input cube (mfs is a cube with nchan=1)
    Output : Cube model image  and/or a list of flux components.   

    
    """   
    
def deconvolve_fast_resolve(img_dataset, decpars):
    """
    An iterative solver to construct a Bayesian model from an observed image(set) and psf(set).
    
    Sky Model : Pixel amplitudes
    Algorithm : Bayesian formulation that includes constraints on the flux distribution and wideband support.
        
    Input : Requires an input cube (mfs is a cube with nchan=1)
    Output : Cube model image, Error map (Spectral index map)

    """   

def deconvolve_rotation_measure_clean(img_dataset, decpars):
    """
    An iterative solver to construct a full-polarization model from an observed image(set) and psf(set).

    Sky Model : Per flux component, delta-functions in lambda-square space
    Algorithm : 
        Step (1) : Transform the cube to lambda-square space
        Step (2) : Construct a 3D RM-synthesis PSF
        Step (3) : Run CLEAN based-deconvolution
        Step (4) : Transform back to frequency space.
   
    Input : Requires an input cube (mfs is a cube with nchan=1)
    Output : Cube model image, Error map (Spectral index map)

    """   


def restore_model(img_dataset, restorepars):
    """
    Restore a deconvolved model.
    
    Inputs : target resolution could be native or 'common' or explicitly specified.
    
    Cube and single-term imaging : 
    - Smooth the model image (Jy/pixel) to the target resolution
    - Smooth the residual image (Jy/beam) to the target resolution
    - Add the two smoothed images
    
    Multi-term imaging :
    - Smooth the model taylor coefficient images to the target resolution
    - Apply the inverse Hessian to the residual image vector (data-space to model-space)
      (At non-native target resolution, also compute a new Hessian matched to the scale of the restoring beam.)
    - Smooth the model-space residuals to the target resolution
    
    Re-restoration may be done simply by calling this same method again with a different target resolution. 
    Calculations will start with the native model and residual images. 
    Note that re-restoration with cngi.image.imsmooth() will not be accurate for multi-term imaging. 
    """   

    
def make_sd_residual_image():
    """
    Construct an observed single dish image cube from single-dish data
    
    If a model image is supplied, implement : 
        Residual image = Observed image - { Model image (conv) SD PSF }
    
    """   
# JW The substraction should be done by the user. make_sd_image
    
def make_sd_point_spread_function():
    """
    Construct a single dish PSF image cube, containing the effective SD beam per frequency.
    """   
def make_sd_weight_image():
    """
    Construct a single dish weight map that illustrates the observing pattern of the mosaic. 
    """   
    
def feather(img_dataset_lowres, img_dataset_highres):
    """
    Feather two images together, based on restoring beam information stored in both. 
    
    Output image = iFT( FT(lowres_image) + [1-FT(lowres_beam)] x FT(highres_image) )
    
    TBD : Do this for the entire image_set (psf, image) and updated restoring-beam information as well ? 
    
    """   
    
def linear_mosaic(img_datasets, img_mosaic):
    """
    Construct a linear mosaic as a primary-beam weighted sum of a set of input images. 
    Individual images are re-sampled onto a larger image grid and summed.
       
    Assume flat-noise normalization for the inputs.  ( TBD : Or flatsky? )
    
    Output image :  sum( input_images ) / sum ( input_pbs )
    
    TBD : This requires some sort of merging of img_datasets. 
          ? CNGI demo on how to append/add images to an image_set and ensure that meta-data are consistent ?
    
    """   

### Imaging Utils

In [0]:
def _normalize(direction, normtype):
    """
    PB normalization on the cubes 
    
    direction : 'forward''reverse'
    normtype : 'flatnoise','flatsky','common','pbsquare'
    
    Multiply and/or divide by PB models, accounting for masks/regions.
    
    """
def _grid_cube(type={'psf','obs','res','pbweight'}):
    """
    Do the gridding

    Use the Gridding convolution function from the cache, and apply phase-gradients for pointing offsets.

            Pointing Offset : Phase gradient across the GCF (from pointing meta-data or pointing_cal_dataset)
                    ( Include support for Heterogeneous Arrays where pointing offset varies with antenna
                      Include support for time-varying offsets : read from Pointing meta-data)

            Mosaic Offset : Phase gradient across the GCF + UVW-rotation to move data to image phasecenter.
    
    Note : A cube with one channel = mfs with nterms=1 
    
    (TBD : This method has to be call-able from outside of imager, for example, for auto-flagging. Where to locate it ?).
    """
    
def _degrid_model_cube():
    """
    Do the de-gridding
    Note : A cube with one channel = mfs with nterms=1 

    """
    
# JW I have not created these files since they are not part of the api

### Methods to move down into CNGI.image

In [0]:
def cube_to_mfs(img_dataset, nterms):
    """
    Collapse a cube to a continuum image (set)
    
    Based on 'nterms', evaluate Taylor-weighted sums across frequency. 
    (In casa6, casatasks.sdintimaging contains a python cube_to_taylor() implementation )
    
    To be used as a convertor during image reconstruction, and also stand-alone.
    
    """   

def mfs_to_cube(img_dataset, nterms):
    """
    Expand a continuum image to a cube
    
    Based on 'nterms', evaluate a Taylor polynomial across frequency in the output cube.
    (In casa6, casatasks.sdintimaging contains a python cube_to_taylor() implementation )
    
    To be used as a convertor during image reconstruction, and also stand-alone.
    
    """   

def corr_to_stokes(img_dataset):
    """
    Convert from the correlation basis (XX,YY,XY,YX) or (RR,LL,LR,RL) to the Stokes basis (I,Q,U,V)
       
    To be used as a convertor during image reconstruction, and also stand-alone
    """   
    
def stokes_to_corr(img_dataset):
    """
    Convert from Stokes basic (I,Q,U,V) to a specified correlation basis (XX,YY,XY,YX) or (RR,LL,LR,RL)
       
    To be used as a convertor during image reconstruction, and also stand-alone    
    """   

    
def fourier_transform(img_dataset):
    """
    A 2D Fourier transform
    
    Numbers : np.fft.fftshift( np.fft.fftn(np.fft.ifftshift( input_array ))  )
    
    Image Coordinate System : Convert as appropriate to 'UV' coordinates with uv-cell size. 

    """   
# JW not yet added to cngi

### Imaging Examples

These examples relate to the different types of images listed in https://casa.nrao.edu/casadocs/casa-5.0.0/synthesis-imaging/image-definition

 

#### Cube and Continuum (narrow-field, wide-field and joint-mosaic)
Example : Cube or Continuum imaging (nterms=1 and nterms>1) for narrow-field, wide-field and joint mosaic imaging, including visibility pre-processing for topo-lsrk conversion and automasking.  This is a pipeline imaging use-case.

In [0]:
#--------------------------------------------------------------------- Data Selection
# Construct a selected vis dataset
vis_dataset = cngi.dio.read_vis(visname, selpars)
# JW cngi.dio.read_vis does not do selection. I don't know how necessary this function is. All it does is wrap open_zarr()


# -------------------------------------------------------------- Visibility Preprocessing
# topo->lsrk + channel binning + ephemeris sources
cngi.vis.regridspw(vis_dataset, impars)
# Apply flags for all future steps
cngi.vis.applyflags(vis_dataset)
# Phasecenter rotation
cngi.vis.rotateuvw(vis_dataset) # TBD : Here, or inside _make_grid ?  


#--------------------------------------------------------------------- Image Definition
# Construct an empty image set
img_dataset = cngi.dio.write_image(impars)
# Set image weighting scheme 
ngcasa.imaging.make_imaging_weight(img_dataset, weightpars)
# Define gridding convolution functions. Aterm, Wterm, JointMosaic are specified here.
ngcasa.imaging.make_gridding_convolution_function(img_dataset, gridpars)

#--------------------------------------------------------------------- Make initial images
# Make PSF
ngcasa.imaging.make_psf(img_dataset, vis_dataset, gridpars)
# Make PB
ngcasa.imaging.make_pb(img_dataset,vis_dataset,gridpars)
# Make Residual image and normalize it
ngcasa.imaging.make_residual_image(img_dataset,vis_dataset, gridpars, normpars)

#------------------------------------------------------------------ Iteration Control
# Initialize the mask
ngcasa.imaging.make_mask(img_dataset,maskpars)
# < Interactive Clean GUI >
# Check convergence criteria
iter_rec = ngcasa.imaging.is_converged(img_dataset, iterpars, None)


# Perform iterative reconstruction
while( iter_rec['stopcode']=='continue' ):
    # -----------------------------------------------------------------------Minor cycle
    exec_rec = ngcasa.imaging.deconvolve_point(img_dataset, decpars, iter_rec)
    #-----------------------------------------------------------------------------------
    
    # -----------------------------------------------------------------------Major cycle
    # Model prediction
    ngcasa.imaging.predict_modelvis_image(img_dataset, vis_dataset, normpars, gridpars)
    # Make residual image and normalize it.
    ngcasa.imaging.make_residual_image(img_dataset,vis_dataset, gridpars,normpars)
    #-----------------------------------------------------------------------------------

    #------------------------------------------------------------------Iteration Control
    # Update the mask
    ngcasa.imaging.make_mask(img_dataset,maskpars)
    # < Interactive Clean GUI >
    # Check convergence criteria
    iter_rec = ngcasa.imaging.is_converged(img_dataset, iterpars, exec_rec)
    #-----------------------------------------------------------------------------------

    
#------------------------------------------------------------------    Restoration
# Restore the model image
ngcasa.imaging.restore_model(img_dataset)


#### Other imaging algorithms (multi-term, rm-synthesis)

Wide-band multi-term imaging with wideband pb-correction may be run by setting up a cube major cycle followed by deconvolve_multiterm.

Rotation-measure synthesis may be called by setting up a full-stokes cube major cycle, followed by a deconvolve_rotation_measure_clean

In both these cases, the deconvolution algorithm starts with an image cube, transforms the image into the sky model space (sparse basis), performs the deconvolution in that space, and transforms the model back to the cube in preparation for the next major cycle.   Wideband Primary beam correction (for Stokes I) is done simply by ensuring 'flatsky' or 'commonpb' normaliation at the end of the make_residual step. 

In [0]:
# Prepare the residual image cube : Normalize to a common pb. This implements widebandpbcor. 
ngcasa.imaging.make_residual_image(img_dataset,vis_dataset, gridpars, normtype='commonpb')

# Start iterative deconvolution...

    # Run the deconvolver (input : residual and psf cubes, output : model cube)
    ngcasa.imaging.deconvolve_multiterm_clea(img_dataset)

    # Model prediction
    ngcasa.imaging.predict_modelvis_image(img_dataset, vis_dataset, normpars, gridpars)

    # Make residual image and normalize gain
    ngcasa.imaging.make_residual_image(img_dataset,vis_dataset, gridpars,normtype='commonpb') 

#### Interactive Clean

There are two parts to interactive image reconstruction.

(1) Mask drawing/viewing : Use a GUI to interactively draw a mask or to simply visualize the current mask. This may be used in conjunction with the ngcasa.imaging.make_mask() to view and/or edit the resulting region.

(2) Editing iteration control parameters at run-time : The same GUI used for mask visualization may be used to display and accept edited values for user-parameters. 

In the above example, this interactive step would reside in between 'Update Mask' and 'Check convergence criteria'. 

See https://gitlab.nrao.edu/rurvashi/interactive-imaging-with-casa6  for an example (using casa6) of how this may be achieved via a stand-alone call to a GUI in-between the major and minor cycles. Convergence history may also be displayed at this stage, allowing for an interactive user to decide if iteration-control parameters should change or not. 

#### Linear Mosaics and Joint mosaics

Three options exist.

(1) The gridder allows for joint mosaic phase gradients to be applied to gridding convolution functions. No change to the code shown above. This is equivalent to mosweight=False in casa6.casatasks.tclean()

(2) ngcasa.imaging.linear_mosaic may be used to combine restored images from different pointings (or clusters of joint-mosaic pointings). This is a post-deconvolution step.

(3) Use cngi.image.linear_mosaic() to calculate a weighted sum of images from different pointing subsets, in-between the major and minor cycles. 
- This option has the advantage of allowing smaller image sizes for individual gridder calls.
- This implicitly implements 'mosweight=True' of casa6.casatasks.tclean() because each pointing (or subset of pointings) is gridded and normalized separately. 

TBD : Will we get (3) by using (1) but just choosing the partition axis to be along fields ? Almost, but no imsize reduction, and it will always be mosweight=False. 

Below is an implementation of (3). TBD : Can this be simplified ? 



In [ ]:
# JW If we add the field as an image and grid dimension this can be simplified (d0,d1,wstack,field,chan,pol). The will then require two more grid parameters.

In [0]:
## Image Reconstruction with Linear Mosaics before deconvolution.
## Major cycle runs separately for each pointing (or subset of pointings)
## Minor cycle runs on a joint image.

img_datasets={}

for imfield in list_of_fields:
    
    # Construct a selected vis dataset for one pointing (or subset of pointings)
    vis_dataset = cngi.dio.read_vis(visname, selpars)

    ### Code blocks from above for visibility preprocessing, image definition, make_psf, make_pb. 
    cngi.vis.regridspw(vis_dataset, impars)
    cngi.vis.applyflags(vis_dataset)
    cngi.vis.rotateuvw(vis_dataset) # TBD : Here, or inside _make_grid ? 
    img_dataset = cngi.dio.write_image(impars)
    ngcasa.imaging.make_imaging_weight(img_dataset, weightpars)
    ngcasa.imaging.make_gridding_convolution_function(img_dataset, gridpars)
    ngcasa.imaging.make_psf(img_dataset, vis_dataset, gridpars)
    ngcasa.imaging.make_pb(img_dataset,vis_dataset,gridpars)    
    
    # Make Residual image and normalize it (normalizing per pointing => mosweight=True)
    ngcasa.imaging.make_residual_image(img_dataset,vis_dataset, gridpars, normpars)
    
    # Accumulate image datasets for each pointing (or subset of pointings)
    img_datasets[imfield] =  img_dataset
    
# Do a linear mosaic to generate the image to send to the minor cycle
cngi.image.linear_mosaic(img_datasets, img_linmos)

# Setup iteration control and masks
ngcasa.imaging.make_mask(img_linmos,maskpars)
iter_rec = ngcasa.imaging.is_converged(img_linmos, iterpars, None)

# Perform iterative deconvolution
while( iter_rec['stopcode']=='continue' ):
    exec_rec = ngcasa.imaging.deconvolve_point(img_linmos, decpars, iter_rec)

    for imfield in list_of_fields:
        # Regrid the linmos model image onto subset model images
        cngi.image.regrid(img_linmos, img_datasets[imfield])
        # Model prediction
        ngcasa.imaging.predict_modelvis_image(img_datasets[imfield], vis_dataset, normpars, gridpars)
        # Make residual image and normalize it.
        ngcasa.imaging.make_residual_image(img_datasets[imfield],vis_dataset, gridpars,normpars)

    # Do a linear mosaic to generate the image to send to the minor cycle
    cngi.image.linear_mosaic(img_datasets, img_linmos)
    
    # Update iteration control and masks
    ngcasa.imaging.make_mask(img_dataset,maskpars)
    iter_rec = ngcasa.imaging.is_converged(img_dataset, iterpars, exec_rec)

    
#------------------------------------------------------------------    Restoration
# Restore the model image
ngcasa.imaging.restore_model(img_dataset)


#### Multi-field Imaging

Purpose : To image outlier sources in separate small images in addition to the main large image. 

Use-Cases : 
- A bright outlier source far from the region of interest would cause the image size to greatly increase if imaged as part of a single image. This single large image would also likely be mostly empty. 
- A bright outlier source requires a different gridding or deconvolution algorithm from the main field, but must be part of the same reconstruction run. A continuum detection experiment at the center of the field would require nterms=1 with (say) multiscale clean, but a bright point source at the half-power level of the primary beam has strong spectral structure induced by the primary beam. In this case, there is no interest in flux accuracy of the bright outlier source, but it must be modeled and subtracted. A multi-term point-source deconvolution algorithm with the standard gridder may be used on the outlier source while a multi-scale nterms=1 imaging run is done on the main field. 

Algorithm Steps : 
  - (1) The same data are gridded onto multiple uv-grids to form a list of observed images and psfs. 
  -  (2) Each such image_field is deconvolved separately in the image domain.  
  -  (3) Iteration control must be merged across image_fields. The return dicts from ngcasa.imaging.has_converged() for each image_field must be merged before parameters are sent to the individual deconvolvers. 
  -  (4) Model images from all image_fields are reconciled to handle overlap regions.
  (In case of overlap, use the specified order of the input image_fields to indicate precedence and to blank out overlapping model image pixels for all but one image_field. Or, apply weights. )
  -  (5) Predict model visibilities separately for each image_field, adding to the model_data array in the vis_dataset.  

Steps 1,2,5 are done independently per image_field. 
Steps 3 and 4 implement the desired relation between these fields for iteration-control and in model-prediction. 

Below is an implementation. TBD : Can this be simplified ?

In [0]:
## Image Reconstruction with Multi-Field imaging
## Major cycle runs once, using models predicted from all image fields.
## Minor cycle runs separately on each image-field

# Construct a selected vis dataset
vis_dataset = cngi.dio.read_vis(visname, selpars)
# Visibility pre-processing : topo->lsrk + channel binning
cngi.vis.regridspw(vis_dataset, impars)
# Apply flags for all future steps
cngi.vis.applyflags(vis_dataset)

# Construct a list of empty image sets
img_datasets={}
for i in range(N_field):
    img_datasets[i] = cngi.dio.write_image(impars[i]) 

# Set image weighting scheme 
ngcasa.imaging.make_imaging_weight(img_datasets[0], weightpars)
# Define gridding convolution functions. Aterm, Wterm, JointMosaic are specified here.
ngcasa.imaging.make_gridding_convolution_function(img_dataset, gridpars)

iter_recs={}
for i in range(N_field):
    # Make PSF
    ngcasa.imaging.make_psf(img_dataset[i], vis_dataset, gridpars)
    # Make PB
    ngcasa.imaging.make_pb(img_dataset[i],vis_dataset,gridpars)
    # Make Residual image and normalize it
    ngcasa.imaging.make_residual_image(img_dataset[i],vis_dataset, gridpars, normpars)

    # Initialize the mask and iteration control
    ngcasa.imaging.make_mask(img_dataset[i],maskpars)
    iter_recs[i] = ngcasa.imaging.is_converged(img_linmos, iterpars, None)


# Merge iteration control rules/parameters across fields
## Implement logic here to reconcile all iter_recs[] into a single iter_rec to apply to all fields. 
    
# Perform iterative reconstruction
while(iter_rec['stopcode']=='continue'):
    for i in range(N_field):
        ngcasa.imaging.deconvolve(img_dataset[i], decpars)

    # Implement code to handle overlapping regions in the model list. 
    # Use list ordering to pick only the first model and blank overlapping regions in other model images   

    ## Model prediction (incremental additions)
    for i in range(N_field):
        ngcasa.imaging.predict_modelvis_image(img_dataset[i], vis_dataset, normpars, gridpars, incremental=True)

    for i in range(N_field):
        ## Make residual image and normalize it.
        ngcasa.imaging.make_residual_image(img_dataset[i],vis_dataset, gridpars,normpars)

        # Update the mask
        ngcasa.imaging.make_mask(img_dataset[i],maskpars,interactive=T/F)
    
# Restore the model images
for i in range(N_field):
    ngcasa.imaging.restore_model(img_dataset[i])

#### Single Dish Imaging with Deconvolution
Purpose : To remove the effect of the Single Dish effective beam from the observed images.

Algorithm Steps : 

In [0]:
## Open datasets and pre-process as needed

# Make the observed image
ngcasa.imaging.make_single_dish_residual(img_dataset)
# Make the PSF
ngcasa.imaging.make_single_dish_psf(img_dataset)
    
# Initialize the mask and iteration control
ngcasa.imaging.make_mask(img_dataset,maskpars)
iter_rec = ngcasa.imaging.is_converged(img_dataset, iterpars, None)

# Perform iterative reconstruction
while( iter_rec['stopcode']=='continue' ):
    # Minor Cycle
    exec_rec = ngcasa.imaging.deconvolve_multiterm clean(img_dataset, decpars, iter_rec) # MSClean or MSMFS.

    # Major Cycle 
    ngcasa.imaging.make_single_dish_residual(img_dataset)
    
    # Update the mask and iteration control
    ngcasa.imaging.make_mask(img_dataset,maskpars)
    iter_rec = ngcasa.imaging.is_converged(img_dataset, iterpars, exec_rec)

# Restore the model image
ngcasa.imaging.restore_model(img_dataset)

## This example does not use the sd_weight_image(). TBD : Update to use it

#### Joint Single Dish and Interferometer Imaging

Purpose : To use constraints from both INT and SD datasets during a joint reconstruction. 

Algorithm Steps : 

(1) Construct Cube PSFs and Residual Images from INT and SD datasets separately.

(2) Apply cngi.image.feather() to merge them and produce a new img_set containing joint information.

(3) Minor cycle : ngcasa.imaging.deconvolve()

(4) Iteration control and masking : Same as interferometer imaging

(5) Major cycle
     - For INT, follow the same process as in the example above
     - For SD, Residual image = Observed image - { Model image (conv) SD PSF }
     Call cngi.image.feather() to merge the new residual images.
     

In [0]:
## Open datasets and pre-process as needed

# Make the observed SD image
ngcasa.imaging.make_single_dish_residual(sd_imset)
# Make the SD PSF
ngcasa.imaging.make_single_dish_psf(sd_imset)

# Make the INT PSF
ngcasa.imaging.make_psf(int_imset, vis_dataset, gridpars)
# Make the INT PB
ngcasa.imaging.make_pb(int_imset, vis_dataset,gridpars)
# Make INT Residual image and normalize it
ngcasa.imaging.make_residual_image(int_imset, vis_dataset, gridpars, normpars)

    
# Feather the SD and INT Cubes together ( PSF and Residual )
joint_imset = ngcasa.imaging.feather(sd_dataset, int_dataset, 'psf')
joint_imset = ngcasa.imaging.feather(sd_dataset, int_dataset, 'residual')
    
# Initialize the mask and iteration control
ngcasa.imaging.make_mask(joint_imset,maskpars)
iter_rec = ngcasa.imaging.is_converged(joint_imset, iterpars, None)

# Perform iterative reconstruction
while( iter_rec['stopcode']=='continue' ):
    # Minor Cycle
    exec_rec = ngcasa.imaging.deconvolve_multiterm(joint_imset, decpars, iter_rec) # MS-Clean or MSMFS

    # Copy/transfer the joint model from joint_imset to sd_imset and int_imset
    
    # Major Cycle for SD
    ngcasa.imaging.make_single_dish_residual(sd_imset)
    # Make INT Residual image and normalize it
    ngcasa.imaging.make_residual_image(int_imset, vis_dataset, gridpars, normpars)
    # Feather the SD and INT Cubes together ( PSF and Residual )
    joint_imset = ngcasa.imaging.feather(sd_dataset, int_dataset, 'residual')
    
    # Update the mask and iteration control
    ngcasa.imaging.make_mask(joint_imset,maskpars)
    iter_rec = ngcasa.imaging.is_converged(joint_imset, iterpars, exec_rec)

# Restore the model image
ngcasa.imaging.restore_model(joint_imset)


### Saving model visibilities

Model prediction and saving comes in several flavors. 
(1) From a component list
(2) From a model image
(3) uv-cont-fit

Model prediction prior to calibration will require the target array to be 'model' whereas simulation will require the target array to be 'data'. ngcasa.imaging.predict_modelvis_xxx() methods take a parameter to specify target array. 

(1) Calculate visibilities from flux-component lists. They may be observatory calibrator models, or the outputs of an imaging algorithm that produces component lists (e.g. ASP)

In [0]:
# Construct a selected vis dataset
vis_dataset = cngi.dio.read_vis(visname, selpars)

# Preprocess for freq-frame conversions
cngi.vis.regridspw(vis-dataset)  # Convert from topo to lsrk

# Predict model visibilities ( in lsrk frame )
ngcasa.imaging.predict_modelvis_component(vis_dataset, component_list)

# Undo frame conversions
cngi.vis.regridspw(vis_dataset)  # Convert from lsrk back to data frame (topo)

# Write to disk
cngi.dio.write_zarr(vis_dataset)
    
# JW using the storage_parm design makes this unnecessary

(2) Calculate visibilities from images, by degridding. The images may be observatory calibrator models or the output of image reconstrunction.

In [0]:
# Construct a selected vis dataset
vis_dataset = cngi.dio.read_vis(visname, selpars)

# Preprocess for freq-frame conversions
cngi.vis.regridspw(vis_dataset)  # Convert from topo to lsrk

# Set up de-grid options for PSterm, Aterm, Wterm, JointMosaic are specified here.
ngcasa.imaging.make_gridding_convolution_function(img_dataset, gridpars)

# Predict model visibilities ( in lsrk frame )
ngcasa.imaging.predict_modelvis_image(vis_dataset, img_dataset)

# Undo frame conversions
cngi.vis.regridspw(vis_dataset)  # Convert from lsrk back to data frame (topo)

# Write to disk
cngi.dio.write_zarr(vis_dataset)


    
(3) Baseline-based continuum model fitting ( used in UV-continuum subtraction )

In [0]:
# Construct a selected vis dataset
vis_dataset = cngi.dio.read_vis(visname, selpars)

# Preprocess for freq-frame conversions and phase rotation to get the source at the phasecenter
cngi.vis.regridspw(vis_dataset)  # Convert from topo to lsrk
cngi.vis.rotateuvw(vis_dataset)  # Rotate to get source at observatory phase center

# Do the uvcontfit : Writes the 'model' array in the XDS vis_dataset
cngi.vis.uvcontfit(vis_dataset) # Fit a continuum model per baseline (valid only for a point source at phasecenter)

# Undo frame conversions and rotations
cngi.vis.rotateuvw(vis_dataset)
cngi.vis.regridspw(vis_dataset)  # Convert from lsrk back to data frame (topo)

# Write to disk
cngi.dio.write_zarr(vis_dataset)

### UV-Continuum Subtraction

There are three ways of implementing uv-continuum subtraction. They follow the three options described above for model prediction and saving.  Insert a  cngi.vis.uvsub() step just prior to the final write to zarr. 

In [0]:
## Pick one of the model prediction methods from above.
vis_dataset = cngi.dio.read_vis(visname, selpars)
cngi.vis.regridspw(vis_dataset)  # Convert from topo to lsrk
cngi.vis.rotateuvw(vis_dataset)  # Rotate to get source at observatory phase center
cngi.vis.uvcontfit(vis_dataset) # Fit a continuum model per baseline (valid only for a point source at phasecenter)
cngi.vis.rotateuvw(vis_dataset)
cngi.vis.regridspw(vis_dataset) 

# Subtract the model from the data. Specify parameters to uvsub to pick which array names to use
cngi.vis.uvsub(vis_dataset)

# Write to disk
cngi.dio.write_zarr(vis_dataset)

### Pointing Self-Calibration

Pair a pointing-offset solver from the calibration module 
with model prediction via A-Projection de-gridding. 
Incorporate the solve step within the imaging major cycle such that update pointing solutions (in a cal_dataset) are applied during the subsequent residual gridding step. 

TBD : Check algorithm with listing in publication...



In [0]:
## Major Cycle of image reconstruction, for pointing self-calibration

# Set up de-grid options for Aterm with pointing offsets read from a pointing dataset.
ngcasa.imaging.make_gridding_convolution_function(img_dataset, gridpars)

# Model prediction, using current values from the pointing dataset
# This fills in the 'MODEL' array in the vis_dataset
ngcasa.imaging.predict_modelvis_image(img_dataset, vis_dataset, pointing_cal_dataset, normpars, gridpars)

# Solve for pointing offsets
pointing_cal_dataset = ngcasa.calibration.solve_pointing(vis_dataset)

# Make residual image and normalize it.
ngcasa.imaging.make_residual_image(img_dataset,vis_dataset, pointing_cal_dataset, gridpars,normpars)
#-----------------------------------------------------------------------------------




## Calibration
--------- This section is very preliminary, 
with only the most basic calibration operations illustrated. 



### Calibration API


In [0]:
def solve(vis_dataset, solpars):
    """
    Calculate antenna gain solutions according to the parameters in solpars.
    The input dataset has been pre-averaged/processed and the model visibilities exist
    
    Iteratively solve the system of equations g_i g_j* = V_data_ij/V_model_ij  for all ij. 
    Construct a separate solution for each timestep and channel in the input dataset.
    
    Options : amp, phase or both
              solution type (?) G-term, D-term, etc... 
              Data array for which to calculate solutions. Default='DATA'
              
    TBD : Single method with options for solutions of different types ? 
          Or, separate methods for G/B, D, P etc.. : solve_B, solve_D, solve_B, etc...
    
    """


def apply(vis_dataset, [ cal_dataset ], arr_name):
    """
    Apply antenna gain solutions according to the parameters in solpars.
    
    Calculate  V_ij(corrected) = V_ij(observed) / g_i g_j*
    
   
    Inputs : 
        List of calibration solution datasets (to apply in the specified order)
        Interpolation type ? 
        Data array on which to operate. Default='DATA'
        Data array in which to fill the output.  Default='CORRECTED_DATA'
                - this option exists in order to support simulator's corrupt operation where we'd pick 'DATA'
    
    
    TBD : Should this translation of the caltable back to the original un-averaged data be done here, 
          or in a centralized CNGI method that handles such interpolations for all methods that need
          to convert averaged values into un-averaged values.  Note the difference between just copying
          and expanding values, versus interpolation. 
          
    TBD : Single apply() method, or several ? 
    
    """
    

### Calibration Examples



#### bandpass + gaincal : 
Use case : Solve for and applying calibration solutions in a sequence. 

Step (1) : Predict Model

Step (2) Bandpass calibration, where data are averaged across time. Solutions are per channel.

Step (3) : Gain calibration, where data are averaged across frequency. Solutions are per time interval

Also, include an autoflag step on the gain solutions before applying, in Step (1).

In [0]:
# Construct a selected vis dataset
vis_dataset = cngi.dio.read_vis(visname, selpars)

## Step (1) 

# Predict model visibilities (in lsrk frame) from a component list, and write to the DATA array in the XDS
cngi.vis.regridspw(vis_dataset)  # Convert from topo to lsrk
ngcasa.imaging.predict_modelvis_component(vis_dataset, component_list, arr_name='DATA') # De-grid and predict model
cngi.vis.regridspw(vis_dataset)  # Convert from lsrk back to data frame (topo)

## Step (2)

# Do time-averaging
vis_dataset_time_avg = cngi.vis.timeaverage(vis_dataset)

# Solve for gains ('solpars' encodes the solution type)
cal_dataset_bandpass = ngcasa.calibration.solve(vis_dataset_time_avg, solpars, use_arr_name='data')

# <Optionally run autoflag on the gain solutions to take out outliers>
# ngcasa.flagging.auto_clip(cal_dataset_bandpass)
# cngi.vis.applyflags(cal_dataset_bandpass)

# Apply the solutions to the original dataset
ngcasa.calibration.apply(vis_dataset, 
                         cal_dataset_bandpass, 
                         use_arr_name='data', 
                         out_arr_name='corrected_data')

## Step (3) 

# Do chan-averaging on original dataset
vis_dataset_chan_avg = cngi.vis.chanaverage(vis_dataset)

# Solve for gains ('solpars' encodes the solution type)
cal_dataset_gaincal = ngcasa.calibration.solve(vis_dataset_chan_avg, solpars, use_arr_name='corrected_data')

# Apply the solutions to the pre-calibrated dataset. Note the change in arr_name. 
ngcasa.calibration.apply(vis_dataset, 
                         cal_dataset_gaincal, 
                         use_arr_name='corrected_data', 
                         out_arr_name='corrected_data')

## OR..... Re-apply all the solutions to the original data...
#ngcasa.calibration.apply(vis_dataset, 
#                         [cal_dataset_bandpass, cal_dataset_gaincal], 
#                         use_arr_name='data',
#                         out_arr_name='corrected_data')

# Save calibrated data to disk
cngi.write_zarr(vis_dataset)


#### Solutions requiring pre-apply of upstream solutions

Example : Pre-apply a pre-existing bandpass solution before averaging (to the MODEL data) and solve for time-varying gains. 

TBD : Do we need this mode of operation ? 

-- Since calibration solutions are obtained from a point source model (i.e. data / model ), the multiplication of g_i g_j\*  with Model visibilities is the same as dividing g_i g_j\* out of the Data visibilities.

-- Therefore, will the above solution (of applying solutions to the data and corrected_data column in a sequence) suffice ? Note that nothing is written to disk after each apply, but only written once at the very end.

TBD : If this doesn't cover all use cases, and we need an explicit pre-apply, need to define  ngcasa.calibration.pre_apply() to multiply 'MODEL' data with g_i g_j* from a list of input cal_datasets. 



#### Polarization calibration (TBD)
TBD : Add example ?

## Flagging





### Flagging API

In [0]:
def manual_flag(vis_dataset, selpars):
    """
    Define a set of data selection queries to mark as flags.
    
    For each query in the input list, set flag=1 for the intersection of 
    selections along multiple dimensions.
    
    Inputs : 
        (1) list of selection queries
        (2) array name for output flags. Default = FLAG
        
    TBD : How to implement this ? Just a series of vis_dataset.isel(...) calls in sequence ?
        What magic does the framework provide to make this efficient for the several 1000s 
        of selections that are typical for online flags. 
       
       list_sels : [{'time':[76,77,78], 'chan':[6,7,8,12]},
           {'time':[112,113], 'chan':[6,7,56]}]
       
       for sel in list_sels : 
           new_xds = vis_dataset.isel(**sel)
           new_xds.FLAG = 1   ( or equivalent )
           <save them to original vis_dataset ? >
           
           
    Other ideas : 
    -- xarray.ones_like(xds.DATA).where(<some conditions>, other=0)
      
    """
    
def manual_unflag(vis_dataset, selpars):
    """
    Define a set of data selection queries to mark as flags.
    
    Inputs : 
        (1) list of selection queries
        (2) array name for output flags. Default = FLAG    
    """

def shadow(vis_dataset, shadowlimit):
    """
    Flag all baselines for antennas that are shadowed beyond the specified tolerance. 
    
    All antennas in the zarr-file metadata (and their corresponding diameters) 
    will be considered for shadow-flag calculations. 
    For a given timestep, an antenna is flagged if any of its baselines
    (projected onto the uv-plane) is shorter than  radius_1 + radius_2 - tolerance.
    The value of 'w' is used to determine which antenna is behind the other.
    The phase-reference center is used for antenna-pointing direction.
    
    Antennas that are not part of the observation, but still physically
    present and shadowing other antennas that are being used, must be added
    to the meta-data list in the zarr prior to calling this method. 
    
    Inputs : 
        (1) shadowlimit or tolerance (in m)
        (2) array name for output flags. Default = FLAG  
    """

def elevation(vis_dataset):
    """
    Flag data for low elevations
    
    Inputs : 
        (1) tolerance
        (2) array name for output flags. Default = FLAG    
    """
    
def quack(vis_dataset):
    """   
    TBD : Change name ?!! 
    
    Flag the beginning and/or end of scans to account for observation effects
    such as antenna slewing delays. 
    
    Inputs : 
        (1) time-width, beginning or end or both
        (2) array name for output flags. Default = FLAG    
    """


def auto_clip(vis_dataset):
    """
    An autoflag algorithm that clips the result of a specified expression.

    Inputs : 
        (1) algo parameters
        (2) array name for output flags. Default = FLAG   
        (3) array name for input flags. Default = FLAG    

    If a new flag_array is picked for the output, save only 'new' flags.
    They can be merged with pre-existing flags in a separate step
    
    If an existing flag_array is picked for the output, merge with logical OR.
    """
def auto_rflag(vis_dataset):
    """
    An autoflag algorithm that detects outliers via hierarchical MAD statistics
    applied to the visibility data. 
    
    Inputs : 
        (1) algo parameters
        (2) array name for output flags. Default = FLAG    
        (3) array name for input flags. Default = FLAG   

    If a new flag_array is picked for the output, save only 'new' flags.
    They can be merged with pre-existing flags in a separate step
    
    If an existing flag_array is picked for the output, merge with logical OR.
    """
def auto_tfcrop(vis_dataset):
    """
    An autoflag algorithm that detects outliers based on the assumption that the
    time-frequency plane of the visibilities for a sky signal is smooth in comparison to RFI.
    
    Inputs : 
        (1) algo parameters
        (2) array name for output flags. Default = FLAG    
        (3) array name for input flags. Default = FLAG
    
    If a new flag_array is picked for the output, save only 'new' flags.
    They can be merged with pre-existing flags in a separate step
    
    If an existing flag_array is picked for the output, merge with logical OR.
    """
    
def auto_uvbin(vis_dataset):
    """
    An autoflag algorithm that detects outliers on the gridded spatial frequency plane
    (Algorithm prototype exists).
    
    TBD : How can this method call  ngcasa.imaging._make_grid() and also satisfy code structure rules ?
    
    Inputs : 
        (1) algo parameters
        (2) array name for output flags. Default = FLAG    
        (3) array name for input flags. Default = FLAG
    
    If a new flag_array is picked for the output, save only 'new' flags.
    They can be merged with pre-existing flags in a separate step
    
    If an existing flag_array is picked for the output, merge with logical OR.
    """

def extend(vis_dataset): 
    """
    Extend and grow existing flags.
    
    Options : grow-around, extendflags, growtime, growfreq, antint.. etc....
    
    Inputs : 
        (1) algo parameters
        (2) array name for output flags. Default = FLAG    
        (3) array name for input flags. Default = FLAG
        
    If a new flag_array is picked for the output, save only 'new' flags.
    They can be merged with pre-existing flags in a separate step
    
    If an existing flag_array is picked for the output, merge with logical OR.
    """
    
def summary(vis_dataset):
    """
    Return a dictionary containing metrics to assess flagging quality
    
    Type 1 : Flag counts

    Flag ratio = n_flags/n_total along multiple axes and different levels of granularity.
    
    
    Type 2 : Flag validity 
    
    Compare statistics of flagged versus unflagged visibility data
    and define a metric that quantifies the following.
     
      - Flagged data must have a higher mean than unflagged data
      - Unflagged data should follow Gaussian stats
      - Protect against under-flagging (less than 10%) or over-flagging (more than 70%)

    This option is for pipelines or applications that need to auto-tune autoflag parameters
    (An 'autotune' algorithm prototype exists). 

    Example (this is very rudimentary) : 
    score1 = (mean(flagged_data) - mean(unflagged_data))/mean(unflagged_data)
    score2 = ( max(unflagged_data)/ mean(unflagged_data) - 3.0 )
    score3 = (count(flagged)/count(total) - 0.1)*2 +  (count(flagged)/count(total) - 0.7)*2


    Inputs : 
        (1) list of metrics to evaluate and return
        (2) array name for input flags. Default = FLAG    

    """
# JW The summary can be returned as part of the vis_dataset.
def manage_flags(vis_dataset, in_flags=[], out_flag=''):
    """
    A flag manager that applies Boolean logic operators to 
    - merge multiple flag versions into one.   
    - delete flag versions
    - list existing version names
    
    Inputs : 
        (1) List of input flag array names
        (2) array name for output flags. Default = FLAG
        (3) Operation (AND, OR)  TBD : supply the expression, instead of canned options. 
    
    If an existing flag array is named as the output, the contents are over-written.
    """

### Flagging Examples

Flags are stored as Boolean arrays in the zarr and xarray datasets. 

Flag versions are maintained by giving these arrays names. Each flagging method can write to a specified named flag array. 

Applications that use flags will read from the specified flag array name and set corresponding data array values to NaN (using cngi.vis.applyflags()) before proceeding.
See : https://cngi-prototype.readthedocs.io/en/latest/visibilities.html#Flagging

Interactive flag visualization may be done at the application layer by inserting a plotting/visualization step in between any of the calls to ngcasa.flagging methods. Flag versions may be managed (merged, copied, deleted) using the ngcasa.flagging.manage_flags() method. For the inevitable experimentation required to tune autoflag parameters, a named flag array may be created, used, visualized, and then discarded. 



#### Manual and meta-data based flags
Online flags typically consist of many (1000s) of data selection queries that mark regions to be flagged. 

Shadow and elevation flagging is also typically done.

In [0]:
#Construct a selected vis dataset
vis_dataset = cngi.dio.read_vis(visname, selpars)

#Define manual flags using the 'isel' syntax that references keywords in the zarr/xds
# https://cngi-prototype.readthedocs.io/en/latest/visibilities.html#Selection-and-Splitting

list_sels = [{'time':[76,77,78], 'chan':[6,7,8,12]},
             {'time':[112,113], 'chan':[6,7,56]}]

# Set the FLAG to 1 for all points in the union of all selections
ngcasa.flagging.manual_flag(vis_dataset,list_sels, flag_name='FLAG')

# Calculate shadow and elevation flags
ngcasa.flagging.elevation(vis_dataset)
ngcasa.flagging.shadow(vis_dataset)

#### Autoflag with extension and pre-existing flags
This example demonstrates the use-case of extending flags generated only by
the autoflag algorithm, but not all manually-set pre-existing flags. 

This is a use-case currently not possible in the casa6 flagger framework. 

In [0]:
#Construct a selected vis dataset
vis_dataset = cngi.dio.read_vis(visname, selpars)

# Do some manual flagging and save the flags to the FLAG column. 
ngcasa.flagging.manual_flag(vis_dataset, list_sels, out_flag_name='FLAG')

# Run the rflag algorithm using FLAG as the pre-existing flags,
# Write only the new flags into a new FLAG_AUTO array.
ngcasa.flagging.auto_rflag(vis_dataset_avg,algopars,
                           in_flag_name='FLAG',
                           out_flag_name='FLAG_AUTO')

# Extend only the new autoflags, but not all pre-existing flags
ngcasa.flagging.extend(vis_dataset,extendpars, 
                    in_flag_name='FLAG_AUTO',
                    out_flag_name='FLAG_AUTO')

# Now, merge the flags using a logical OR, and save it into the default 'FLAG' array
ngcasa.flagging.manage_flags(vis_dataset,
                             in_flags=['FLAG','FLAG_AUTO'], 
                             out_flag_name='FLAG', 
                             op='or')

## < Visualize the flags by plotting the data with a chosen flagversion > 
## < If unsatified, discard flagversion using a cngi.dio.xxxx step and repeat the above >

## Save vis_dataset to zarr.
cngi.dio.write_zarr(vis_dataset)

#### Autoflag with pre-averaging
This example demonstrates averaging for autoflagging, expanding the flags back to the original dataset. Two manual flag calls are also included, one on the original data and one after averaging.

Flag expansion is done as a regrid operation. 

In [0]:
# Construct a selected vis dataset
vis_dataset = cngi.dio.read_vis(visname, selpars)

# (1) Do some manual flagging and save the flags to the FLAG column. 
ngcasa.flagging.manual_flag(vis_dataset, list_sels_original, out_flag_name='FLAG')

# Now, average the data in both time and frequency prior to running autoflag.
vis_dataset_time_avg = cngi.vis.timeaverage(vis_dataset)
vis_dataset_time_freq_avg = cngi.vis.chanaverage(vis_dataset_time_avg)

# (2) Run the tfcrop algorithm using FLAG as the pre-existing flags
# and save the new flags into the same flag array.
ngcasa.flagging.auto_tfcrop(vis_dataset_time_freq_avg,algopars,
                           in_flag_name='FLAG',
                           out_flag_name='FLAG')

# (3) Manual flags, using meta-data corresponding to the averaged-data.
ngcasa.flagging.manual_flag(vis_dataset_time_freq_avg, list_sels_lowres, out_flag_name='FLAG')

# (4) Manual unflag, using meta-data corresponding to the averaged-data
ngcasa.flagging.manual_unflag(vis_dataset_time_freq, list_sels_lowres_unflag, out_flag_name='FLAG')


# Now, expand the flags back to the original resolution. 
#        Note that Step (1) was already on the original data. 
#        The results of Steps (2) and (3) and (4) will get expanded out. 
#
## TBD : Need a demo at the CNGI level of how to expand flags back to the original data
##       Expansion is a regrid where the same value is repeated across the expanded range.
cngi.vis.regrid(in_xds=vis_dataset_time_freq, out_xds=vis_dataset)

# Save to disk
cngi.dio.write_zarr(vis_dataset)

#### FlagVersion handling for pipelines
Save and restore flag versions, along with dataset selections/splits.

The use case is based on current operations with casa6, where parallelization is implemented by partitioning the data and running operations that set flags, on each subset.  It is TBD whether this use-case is still relevant with CNGI and ngCASA, but here is an example to showcase how this may be achieved. 

TBD : Need input from pipeline group : Does this represent a relevant use case ? 

In [0]:
#Construct several selected vis datasets
vis_dataset_target1 = cngi.dio.read_vis(visname, selpars_target1)
vis_dataset_target2 = cngi.dio.read_vis(visname, selpars_target2)

# Do different operations on the two datasets
cngi.ngcasa.autoflag_rflag(vis_dataset_target1)
cngi.ngcasa.manual_flag(vis_dataset_target2,list_sels_to_flag)

# Regrid/expand the flags back to the original dataset
cngi.vis.regrid(in_xds=vis_dataset_target1, out_xds=vis_dataset)
cngi.vis.regrid(in_xds=vis_dataset_target2, out_xds=vis_dataset)

# Save to disk
cngi.dio.write_zarr(vis_dataset)

## Simulation
Make the MS frame. Calculate all the meta-data and coordinates. 
Use the calibrator and imager modules for the data prediction and corruption.

### Simulation API
Move the API entirely into CNGI ? 

Note : The API definition should mostly follow the casa6.casatools.simulator interface, except for the steps of calculating visibilities. 

In [0]:
def add_meta_data():
    """
    Make an empty dataset, or append to an existing vis_dataset.
    Each call to this method will construct meta-data (UVW values, time values, etc) for one scan/spw_pol/field. 
    
    A dataset containing multiple scans, fields(pointings) or spectral window and pol settings may be 
    constructed by calling this method in a sequence. 
    
    Inputs : 
        - Observation phase center
        - Spectral window and polarization setup parameters (chanwidth, nchan, startchan, etc...)
        - Array configuration
        - Integration length and Time-range of one scan.

    Output : 
        - A new or appended XDS with new/appended meta-data information

    TBD : Split this into smaller methods as needed. 
        Can follow the casa6.casatools.simulator tool interface for all methods required before the actual calculation of visibility values.    
        May be best to make them simulation_utils methods though. 
    
    """
    
def add_noise():
    """
    Add noise to the DATA column of the MS
    
    Options : 
     - Gaussian random noise of specified standard-deviation
     - Noise that accounts for delta-T and delta-NU of the observation. 
     - Noise that accounts for different antenna collecting areas (from dish diameter meta-data)
    
    """

### Simulation examples
Simulation is an application that uses the simulator, calibrator, flagger and imager modules



#### Simulate a wideband mosaic observation with a heterogenous array.
This is an ngVLA and ALMA use-case. 

- Call the simulator methods in a sequence to generate an empty dataset containing only coordinates and metadata
- Call flagger methods to flag shadowed or low-elevation data
- Call imager methods to predict visibilities
- Call calibrator methods to corrupt the data

Assumption : A true-sky model image already exists in img_dataset. A cal_dataset also exist (to apply gain errors)

In [0]:
# List of mosaic pointings
list_fields=[{'ra1':'xxxx', 'dec1':'xxxx'},
             {'ra2':'xxxx', 'dec2':'xxxx'}]
# List of spectral windows with pol specs
list_spws=[{'start':'1GHz','nchan':100, 'width'='10MHz', 'pol'='RR,LL'},
           {'start':'2GHz','nchan':1000, 'width'='1MHz', 'pol'='RR,LL'}]
# List of scans
list_scans=[{'date':'xxx', 'ha_start':'-4h', 'ha_stop':'-3h', 'integration'='1s'},
            {'date':'xxx', 'ha_start':'-2h', 'ha_stop':'-1h', 'integration'='1s'} ]

# Construct meta-data for the observation.
sim_dataset = None
for scan in scans:
    for field in fields:
        for spw in spws:
            sim_dataset = ngcasa.simulation.add_meta_data(sim_dataset, scan, field, spw)

# Flag data for low elevation or shadows
ngcasa.flagging.shadow(sim_dataset)
ngcasa.flagging.elevation(sim_dataset)

# Predict model visibilities (in lsrk frame) from an image, and write to the DATA array in the XDS
cngi.vis.regridspw(sim_dataset)  # Convert from topo to lsrk
ngcasa.imaging.make_gridding_convolution_function(img_dataset, gridpars) # Setup de-gridding (include het-array PBs)
ngcasa.imaging.predict_modelvis_image(sim_dataset, component_list, arr_name='DATA') # De-grid and predict model
cngi.vis.regridspw(sim_dataset)  # Convert from lsrk back to data frame (topo)

# Corrupt by antenna gains 
ngcasa.calibration.apply(sim_dataset, cal_dataset_corrupt, use_arr_name='DATA', out_arr_name='DATA')

# Add Gaussian random noise
ngcasa.simulation.add_noise(sim_dataset)

# Save to disk
cngi.write_zarr(sim_dataset)

# Questions for CNGI/ngCASA team discussion : 

(1)  Are img_datasets expected to contain one image at a time, or entire sets used for imaging ? 
    - Some methods need to see sets of associated images, whereas others need just one image.
    - Some methods change the shape of the image, but it is still associated with the same reconstruction run
    ==> It seems best to restrict an img_dataset to contain only one image array. 
    
    Example : Cube vs MFS have different image definitions. 
    For mtmfs, major cycles are cubes, and minor cycles are MTMFS. 
    So here, should both definitions reside inside the same img_dataset 
    and the deconvolver know to look for some naming convention ?  
    How to do this cleaner ? 
    
    # JW I think img_dataset can contain any number of images (same for vis_datasets and data). The user should just tell the ngcasa function what image to use (see ngcasa.imaging.make_image api).
    
(2) How to reconcile the 'stateless function' approach with the need for maintaining iteration control state right through an image reconstruction run ? => The application layer will have to manage return-vals, inp_vals and 'state' ?

    # JW States can be attached in the attribute section of the relevant dataset.

(3) When to use parameters to a single method, versus having multiple methods ?  The former allows for cleaner application code. The latter allows for more atomic code structure.  => Decide on a case-by-case basis ? 

    # JW It should be a balance (case-by-case basis). I think the functions you suggested achieve that. 

(4) Are there rules for what data-array names are allowed in the XDS and Zarr datasets ? Can we implement 'versioning' of arrays simply by picking different names ? E.g. Versions of corrected_data or flags. This is something in the MS V3 that we have needed for a long time.    All the above usage examples assume this is possible. 
    
    # JW Any name should be allowed with defaults.

(5) Need more examples in CNGI

- Example of how to merge img_datasets that may have different image shapes ?  Join operation for xds
  - Needed for linear_mosaic, merge_models_across_image_fields...
  
        #JW Maybe a cngi_function should do this.
  
- Example of how to merge vis_Datasets ? Join operation for xds
  - Needed for pipeline use case of doing different operations on different subsets of the data, and then joining the results.  A simple 'regrid' call ? 
  
        #JW Not sure.
  
- MS-Selection replacement examples : How to connect the zarr and xarray metadata to selection data

       #JW Selection should be done using isel etc. There are examples in cngi.

- ComponentList replacement examples : Just a dictionary that we have to define and decide upon ?

        #JW Yes

- Reverse operation for time/chan average or rebin/regrid, or topo-lsrk. 
  E.g. vis.timeaverage () followed by 'edit flags' and then write back to original dataset ? FLAG value is to be copied during expansion. 
  E.g. Caltable solutions need an interpolation to get back to the original data resolution. Interpolation+Extrapolation
  E.g. vis.chanaverage() followed by 'edit CORRECTED_DATA' (e.g. uvcontfit+uvsub), and then write back to original dataset. This makes no sense. So, how to prevent this ?


- Visualization ? 
  - Point to available python libs for data visualization...
  - With array 'versioning' implemented as separate arrays... it's easy to plot/explore versions. On-the-fly application of operations (averaging, coord-shifts, cal-apply, model-predict) is also possible simply by not saving to disk before visualization. 
  - It can be inserted between any sequence of steps in any user application script.
  - Will apply to vis and cal datasets, and img datasets too. This will get us scatter and raster displays for all kinds of datasets...
  
      
